#### Weather Station Analysis Module
author: Ali Hanks

This is an example of importing data from a personal Weather Underground 
weather station setup following the instructions here:
    https://radwatch.berkeley.edu/dosenet/weather

In this module, we will be using the station posting data here:
    https://www.wunderground.com/personal-weather-station/dashboard?ID=KCABERKE95

In [11]:
# These first two lines configure the notebook to embed any plots graphed
#   in the body of the notebook
%matplotlib inline
%config InlineBackend.figure_formats=['svg']

# Standard csv and file io python libraries
import csv
import io
import os

# Library for loading data from a webpage (Python 2 and 3 compatible)
from future.standard_library import install_aliases
install_aliases()
from urllib.request import urlopen, Request

# Main python library for mathematical calculations
import numpy as np

# Plotting related python libraries
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Python libraries for manipulating dates and times as objects
from datetime import datetime
from datetime import timedelta
from matplotlib.dates import date2num

Data from Weather Underground for a specific weather station can be requested in CSV format using this format for the url:
https://www.wunderground.com/weatherstation/WXDailyHistory.asp?ID=KCABERKE95&day=2&month=8&year=2017&graphspan=day&format=1

Comma separated values (.CSV) is a filetype that stores tabular data 
in plaintext, and the columns generated by Weather Underground are:
**Time,TemperatureF,DewpointF,PressureIn,WindDirection,WindDirectionDegrees,
WindSpeedMPH,WindSpeedGustMPH,Humidity,HourlyPrecipIn,Conditions,Clouds,
dailyrainin,SolarRadiationWatts/m^2,SoftwareType,DateUTC**

The following function takes the station ID and the date of interest as arguments and processes the data returned by the corresponding url request
and stores and returns the columns of interest as a 2D list of:
**Time,Temperature,Pressure,Windspeed,Humidity,Precipitation,Solar radiation**

For a simple example of reading a csv file from a url web request look at Module 2 - Import Web CSVs

In [9]:
def data_scrap(ID, date):
    #Scrap weather data of given location and given period of time from websites
    #ID is a string contains weather station ID
    #date is a 1 by 3 string array: Month/Date/Year

    data_temp=[] #Stores raw data from the weather station
    str1 = 'https://www.wunderground.com/weatherstation/WXDailyHistory.asp?ID='
    str2 = '&day='
    str3 = '&month='
    str4 = '&year='
    str5 = '&graphspan=day&format=1'
    url = str1+ID+str2+date[1]+str3+date[0]+str4+date[2]+str5
    response = urlopen(url)
    cr=csv.reader(response)
    #cr=csv.reader(io.TextIOWrapper(response))
    for row in cr:
        if len(row)<= 1: continue
        data_temp.append(row)
    
    #Stores data with correct data type (datetime/string/double)
    data = [[0 for i in range(len(data_temp[1][:])-3)] for j in range(len(data_temp))]

    for i in range(len(data_temp)):
        if i == 0:
            data[0][:]=data_temp[0][0:len(data_temp[i][:])-2]
        elif i > 0:
            data[i][0]=datetime.strptime(data_temp[i][0], '%Y-%m-%d %H:%M:%S')
            data[i][1:data_temp[0][:].index('WindDirection')]=tuple(float(list_item) for list_item in data_temp[i][1:data_temp[0][:].index('WindDirection')])
            data[i][data_temp[0][:].index('WindDirection')] = data_temp[i][data_temp[0][:].index('WindDirection')]
            data[i][data_temp[0][:].index('WindDirection')+1:data_temp[0][:].index('Conditions')] = tuple(float(list_item) for list_item in data_temp[i][data_temp[0][:].index('WindDirection')+1:data_temp[0][:].index('Conditions')])
            data[i][data_temp[0][:].index('Conditions'):data_temp[0][:].index('Clouds')+1] = data_temp[i][data_temp[0][:].index('Conditions'):data_temp[0][:].index('Clouds')+1]
            data[i][data_temp[0][:].index('Clouds')+1:len(data_temp[0][:])-2] = tuple(float(list_item) for list_item in data_temp[i][data_temp[0][:].index('Clouds')+1:len(data_temp[i][:])-3])
    
    #Select data for csv file(Date,Temperature, Pressure, Windspeed, Humidity, Hourly Precipitation, and Solar radiation)
    data_csv = [[0 for i in range(7)] for j in range(len(data))]

    for i in range(len(data_temp)):
        data_csv[i][0:2] = data[i][0:2]
        data_csv[i][2] = data[i][3]
        data_csv[i][3] = data[i][6]
        data_csv[i][4:6] = data[i][8:10]
        data_csv[i][6] = data[i][13]
    
    return data_csv

An example main method that uses the data_scrap function defined above to fill lists with all of the data extracted.

**Exercise:** add plotting to this method following Module 3 to generate similar plots for temperature, pressure, and humidity.

In [10]:
def main():
    #Test Case: 2016/5/19
    start = ['5','19','2016']
    location = 'KCABERKE89'
    data = data_scrap(location, start)

    #Import data from weather station
    date = []
    temperature = []
    pressure = []
    windSpeed = []
    humidity = []
    hourPrecep = []
    solarRadiation = []
    line = 0
    for row in data:
        #skip first row (row that specifies fields)
        if line != 0:
            date.append(row[0])
            temperature.append(row[1])
            pressure.append(row[2])
            windSpeed.append(row[3])
            humidity.append([4])
            hourPrecep.append(row[5])
            solarRadiation.append(row[6])
        line += 1

main()